In [1]:
import torch
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
%matplotlib inline

# Overall workflow and training

Data generation/training can be performed by built-in executable `lasdi`. For this example of Burgers 1D equation, you can simply run on command-line terminal:
```
lasdi burgers1d.yml
```

The workflow can be also manually constructed for those who prefer python scripts and for prototyping. Following code snippets show the high-level view of the workflow in the executable `lasdi`.

In [ ]:
import yaml
from lasdi.workflow import initialize_trainer, run_samples, pick_samples

cfg_file = 'burgers1d.yml'
with open(cfg_file, 'r') as f:
    config = yaml.safe_load(f)

trainer, param_space, physics, autoencoder, sindy = initialize_trainer(config)

# generate initial training/test data
pick_samples(trainer, config)
run_samples(trainer, config)
# initial training given training data
trainer.train()

while (trainer.restart_iter < trainer.max_iter):
    if (trainer.restart_iter <= trainer.max_greedy_iter):
        # perform greedy sampling to pick up new samples
        pick_samples(trainer, config)
        # update training data with newly picked samples
        run_samples(trainer, config)

    # train over given training data
    trainer.train()

If you ran the command instead, a restart file is saved at the end of the training, which can be loaded for post-processing:

In [ ]:
# Specify the restart file you have.
filename = 'lasdi_10_01_2024_17_09.npy'

import yaml
from lasdi.workflow import initialize_trainer
from lasdi.param import ParameterSpace

cfg_file = 'burgers1d.yml'
with open(cfg_file, 'r') as f:
    config = yaml.safe_load(f)

restart_file = np.load(filename, allow_pickle=True).item()

trainer, param_space, physics, autoencoder, sindy = initialize_trainer(config, restart_file)

# Post-processing

Load data for post-processing:

In [4]:
coefs = trainer.best_coefs
X_train = trainer.X_train
X_test = trainer.X_test

param_train = param_space.train_space
param_grid = param_space.test_space
test_meshgrid = param_space.test_meshgrid
test_grid_sizes = param_space.test_grid_sizes
n_init = param_space.n_init

n_a_grid, n_w_grid = test_grid_sizes
a_grid, w_grid = test_meshgrid

t_grid = physics.t_grid
x_grid = physics.x_grid
t_mesh, x_mesh = np.meshgrid(t_grid, x_grid)
Dt, Dx = physics.dt, physics.dx

time_dim, space_dim = t_grid.shape[0], x_grid.shape[0]

n_coef = sindy.ncoefs

They can be also loaded directly from restart file:

In [ ]:
coefs = restart_file['trainer']['best_coefs']
X_train = restart_file['trainer']['X_train']
X_test = restart_file['trainer']['X_test']

paramspace_dict = restart_file['parameters']
param_train = paramspace_dict['train_space']
param_grid = paramspace_dict['test_space']
test_meshgrid = paramspace_dict['test_meshgrid']
test_grid_sizes = paramspace_dict['test_grid_sizes']
n_init = paramspace_dict['n_init']

n_a_grid, n_w_grid = test_grid_sizes
a_grid, w_grid = test_meshgrid

physics_dict = restart_file['physics']
t_grid = physics_dict['t_grid']
x_grid = physics_dict['x_grid']
t_mesh, x_mesh = np.meshgrid(t_grid, x_grid)
Dt = physics_dict['dt']
Dx = physics_dict['dx']

time_dim, space_dim = t_grid.shape[0], x_grid.shape[0]
n_coef = restart_file['latent_dynamics']['ncoefs']

## Gaussian-process uncertainty evaluation
We evaluated the uncertainties of latent dynamics coefficients over 2d parameter space, with samples from GP prediction:

In [ ]:
from lasdi.gp import fit_gps
from lasdi.gplasdi import sample_roms, average_rom
from lasdi.postprocess import compute_errors
from lasdi.gp import eval_gp

n_samples = 20
autoencoder.cpu()

gp_dictionnary = fit_gps(param_space.train_space, coefs)

Zis_samples = sample_roms(autoencoder, physics, sindy, gp_dictionnary, param_grid, n_samples)
Zis_mean = average_rom(autoencoder, physics, sindy, gp_dictionnary, param_grid)

X_pred_mean = autoencoder.decoder(torch.Tensor(Zis_mean)).detach().numpy()
X_pred_samples = autoencoder.decoder(torch.Tensor(Zis_samples)).detach().numpy()

avg_rel_error = np.zeros(param_grid.shape[0])
for k in range(param_grid.shape[0]):
    avg_rel_error[k], _ = compute_errors(X_pred_mean[k], physics, X_test[k].numpy())

max_std = np.zeros(param_grid.shape[0])
for k in range(param_grid.shape[0]):
    max_std[k] = X_pred_samples[k].std(0).max()

avg_rel_error = avg_rel_error.reshape([n_w_grid, n_a_grid]).T
max_std = max_std.reshape([n_w_grid, n_a_grid]).T

gp_pred_mean, gp_pred_std = eval_gp(gp_dictionnary, param_grid)

# Visualization

Plot mean and standard deviation of coefficient matrix.
For SINDy of dimension 5, the coefficient matrix has a shape of (6, 5).

In [ ]:
from lasdi.postprocess import plot_gp2d

plot_gp2d(a_grid, w_grid, gp_pred_mean.reshape([n_a_grid, n_w_grid, -1]), gp_pred_std.reshape([n_a_grid, n_w_grid, -1]),
          param_train, param_labels=['a', 'w'], plot_shape=[6, 5])

In [ ]:
from lasdi.postprocess import heatmap2d

heatmap2d(avg_rel_error * 100, a_grid[:, 0], w_grid[0], param_train, 4, param_labels=['a', 'w'], title='GPLaSDI')

In [ ]:
heatmap2d(max_std * 100, a_grid[:, 0], w_grid[0], param_train, 4, param_labels=['a', 'w'], title=r'max$_{(t,x)}\sqrt{V[\tilde{u}_{\xi^*}]}$   ($\times10^{-2}$)')

In [ ]:
from lasdi.postprocess import plot_prediction

a, w = 0.9, 1.07
param = np.array([[a, w]])
true = physics.solve(param[0])
true = true.detach().numpy().reshape([physics.grid_size[0], physics.nt]).T
scale = 1

Z = sample_roms(autoencoder, physics, sindy, gp_dictionnary, param, n_samples)

Z_mean = Z[0].mean(0)
Z_std = Z[0].std(0)

pred = autoencoder.decoder(torch.Tensor(Z)).detach().numpy()
pred_std = pred[0].std(0)

plot_prediction(param, autoencoder, physics, sindy, gp_dictionnary, n_samples, true, scale)